In [ ]:
from scipy import spatial
import pandas as pd

df = pd.DataFrame(data={'c1': [1, 2, 2], 'c2': [3, 4, 1]})

f1 = df.iloc[0, :]
f2 = df.iloc[1, :]

# compute the cosine similarity between the first 2 rows
cosine_sim = 1 - spatial.distance.cosine(f1, f2)
print(cosine_sim)

In [ ]:
import numpy as np

df = pd.DataFrame(data={'user': [1, 1, 2, 2], 'rating': [3, 4, 1, 2]})

def normalize_ratings(df, rating_col="rating", user_col="user"):
    groups = df.groupby(user_col)[rating_col]
    # computes group-wise mean/std and auto broadcasts to individual groups
    mean = groups.transform(np.mean)
    std = groups.transform(np.std)
    return (df[rating_col] - mean) / std

df["rating_normalized"] = normalize_ratings(df)
df.head()

In [ ]:
import numpy as np

df = pd.DataFrame(data={'user': [1, 1, 2, 2], 'rating': [10, 10, 10, 10], 't': pd.to_datetime(["2019-01-01", "2019-01-02", "2019-01-03", "2019-01-04"])})

def cumsum_days(s, duration='D'):
    return s.diff().astype('timedelta64[%s]' % duration).fillna(0).cumsum().values

def decay_ratings(df, decay=1, rating_col="rating", time_col="t"):
    weight = np.exp(-cumsum_days(df[time_col]) * decay)
    return df[rating_col] * weight

half_life_t = 1
decay = np.log(2) / half_life_t
df["rating_decayed"] = decay_ratings(df, decay=decay)

df.head()

In [ ]:
!pip install pyspark

In [ ]:
import pyspark
from pyspark.ml.recommendation import ALS
sc = pyspark.SparkContext('local[*]')

n_iter = 10
rank = 10
l2_reg = 1

als = ALS() \
    .setMaxIter(n_iter) \
    .setRank(rank) \
    .setRegParam(l2_reg)

In [ ]:
model = als.fit(train_data)
